In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

from tensorflow.keras.models import Model



import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
import pandas as pd

Using TensorFlow backend.
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\Anaconda3\lib\site-packages\tensorboard\compat\t

In [2]:
# load data 

BASE_DIR ='D:/agriculture_computer_vision/Agriculture-Vision/train/images/'

rgb= os.path.join(BASE_DIR, 'rgb/')

cloud_shadow = os.path.join(BASE_DIR, 'cloud_shadow')

standing_water = os.path.join(BASE_DIR, 'standing_water')

double_plant = os.path.join(BASE_DIR,'double_plant')

planter_skip = os.path.join(BASE_DIR + 'planter_skip')

water_way = os.path.join(BASE_DIR,'waterway')

weed_cluster = os.path.join(BASE_DIR, 'weed_cluster')





#test dir
TEST_DIR = 'D:/agriculture_computer_vision/Agriculture-Vision/test/images/'
test_set = os.path.join(TEST_DIR, 'smaller_test_set')

VAL_DIR = 'D:/agriculture_computer_vision/Agriculture-Vision/val/images/rgb/'

In [3]:
# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 20
data_augmentation = True
num_classes = 5


input_shape = (256, 256, 3)

In [4]:
# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

In [5]:
n = 3

In [6]:
# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1
# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

In [7]:
# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version) 

In [8]:
train_df = pd.read_csv(r'C:\Users\User\Desktop\agri_vision\final_encoded.csv')

columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

In [9]:
train_df.drop(columns='Unnamed: 0', inplace=True)

In [10]:
#strip train index
train_df['index'] = train_df['index'].str.rstrip('.png')

In [11]:
train_df['index'] = train_df['index'] + '.jpg'

In [12]:
train_df

,index,cloud_shadow,double_planter,planter_skip,standing_water,waterway,weed_cluster
0,11T3V93AF_2280-3416-2792-3928.jpg,0,0,0,0,1,0
1,11T3V93AF_2373-2788-2885-3300.jpg,0,0,0,0,1,0
2,11T3V93AF_2721-2503-3233-3015.jpg,0,0,0,0,1,0
3,11T3V93AF_3083-3751-3595-4263.jpg,0,0,0,0,1,0
4,11T3V93AF_3877-4192-4389-4704.jpg,0,0,0,0,1,0
...,...,...,...,...,...,...,...
12896,ZYI6ZLYNH_9545-12379-10057-12891.jpg,1,0,0,0,0,0
12897,ZYI6ZLYNH_9545-9819-10057-10331.jpg,1,0,0,0,0,0
12898,ZYI6ZLYNH_9556-3534-10068-4046.jpg,1,0,0,0,0,0
12899,ZYI6ZLYNH_9952-10044-10464-10556.jpg,1,0,0,0,0,0


In [13]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.10) # set validation split

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train_df, 
    directory = 'D:/agriculture_computer_vision/Agriculture-Vision/train/images/rgb/',
    x_col = "index",
    y_col = columns,
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="raw",
    target_size=(256, 256),
    subset='training'
) # set as training data



validation_generator = train_datagen.flow_from_dataframe(
    dataframe= train_df,
    directory = 'D:/agriculture_computer_vision/Agriculture-Vision/train/images/rgb/',
    x_col = "index",
    y_col = columns,
    batch_size=32, 
    seed=42,
    shuffle = True,
    class_mode="raw",
    target_size = (256, 256),
    subset = 'validation')

Found 11611 validated image filenames.
Found 1290 validated image filenames.


In [14]:
from keras import regularizers, optimizers

model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(256,256,3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation='sigmoid'))
model.compile(optimizers.Adam(lr=0.0001, decay=1e-6),loss="categorical_crossentropy", metrics=["accuracy"])

In [15]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 254, 254, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 127, 127, 64)     

In [16]:
# checkpoint
filepath="20_epochs_best_adam.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [17]:
STEP_SIZE_TRAIN = train_generator.n//train_generator.batch_size
VAL_STEP_SIZE = validation_generator.n//validation_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = validation_generator,
                    epochs=20, callbacks=callbacks_list
                   )

Epoch 1/20
362/362 [==============================] - 2480s 7s/step - loss: 1.3254 - accuracy: 0.5837 - val_loss: 0.7649 - val_accuracy: 0.7209

Epoch 00001: val_accuracy improved from -inf to 0.72093, saving model to 20_epochs_best_adam.hdf5
Epoch 2/20
362/362 [==============================] - 2526s 7s/step - loss: 1.1718 - accuracy: 0.5855 - val_loss: 1.8893 - val_accuracy: 0.7209

Epoch 00002: val_accuracy did not improve from 0.72093
Epoch 3/20
362/362 [==============================] - 2180s 6s/step - loss: 1.1050 - accuracy: 0.5858 - val_loss: 0.6029 - val_accuracy: 0.7209

Epoch 00003: val_accuracy did not improve from 0.72093
Epoch 4/20
362/362 [==============================] - 2251s 6s/step - loss: 1.0563 - accuracy: 0.5861 - val_loss: 0.4401 - val_accuracy: 0.7217

Epoch 00004: val_accuracy improved from 0.72093 to 0.72171, saving model to 20_epochs_best_adam.hdf5
Epoch 5/20
362/362 [==============================] - 2278s 6s/step - loss: 1.0135 - accuracy: 0.5901 - val_los

In [41]:
model.save("adam_78.h5")

In [29]:
from keras.models import load_model

model = load_model(r'C:\Users\User\Desktop\agri_vision\adam_78.h5')

In [30]:
val_df = pd.read_csv(r'C:\Users\User\Desktop\agri_vision\val_encoded.csv')

columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

In [31]:
val_df.drop(columns='Unnamed: 0', inplace=True)

In [32]:
#strip train index
val_df['index'] = val_df['index'].str.rstrip('.png')

In [33]:
val_df['index'] = val_df['index'] + '.jpg'

In [34]:
val_df

,index,cloud_shadow,double_planter,planter_skip,standing_water,waterway,weed_cluster
0,18X67NLZ2_1219-1091-1731-1603.jpg,1,0,0,0,0,0
1,18X67NLZ2_1219-1603-1731-2115.jpg,1,0,0,0,0,0
2,18X67NLZ2_1219-579-1731-1091.jpg,1,0,0,0,0,0
3,18X67NLZ2_707-1603-1219-2115.jpg,1,0,0,0,0,0
4,1J7JL2PDK_1130-1579-1642-2091.jpg,0,0,0,0,0,1
...,...,...,...,...,...,...,...
4426,ZKV2W31CE_739-4212-1251-4724.jpg,0,0,0,0,0,1
4427,ZKV2W31CE_777-3820-1289-4332.jpg,0,0,0,0,0,1
4428,ZKV2W31CE_887-3539-1399-4051.jpg,0,0,0,0,0,1
4429,ZKV2W31CE_951-2419-1463-2931.jpg,0,0,0,0,0,1


In [35]:
train_datagen = ImageDataGenerator(rescale=1./255) # set validation split

val_generator = train_datagen.flow_from_dataframe(
    dataframe = val_df, 
    directory = 'D:/agriculture_computer_vision/Agriculture-Vision/val/images/rgb/',
    x_col = "index",
    batch_size=1,
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(256, 256),
)

Found 4431 validated image filenames.


In [36]:
preds = model.predict(val_generator)

In [ ]:
preds

In [37]:
y_val_true = val_df.drop(columns='index').values.argmax(axis=1)
y_val_pred = preds.argmax(axis=1)

In [38]:
# Importing the metrics package from sklearn library
from sklearn import metrics
# Creating the confusion matrix
cm = metrics.confusion_matrix(y_val_true, y_val_pred)

In [39]:
# Assigning columns names
cm_df = pd.DataFrame(cm, 
            columns = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster'],
            index = ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster'])
# Showing the confusion matrix
cm_df

,cloud_shadow,double_planter,planter_skip,standing_water,waterway,weed_cluster
cloud_shadow,146,10,0,13,5,35
double_planter,2,251,0,4,11,172
planter_skip,2,12,0,0,0,9
standing_water,6,6,0,123,7,147
waterway,4,15,0,4,178,310
weed_cluster,74,64,0,32,117,2672


In [40]:
metrics.accuracy_score(y_val_true, y_val_pred)

0.7605506657639359

In [44]:
target_names =  ['cloud_shadow', 'double_planter', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']

from sklearn.metrics import classification_report
print(classification_report(y_val_true, y_val_pred, target_names=target_names))

                precision    recall  f1-score   support

  cloud_shadow       0.62      0.70      0.66       209
double_planter       0.70      0.57      0.63       440
  planter_skip       0.00      0.00      0.00        23
standing_water       0.70      0.43      0.53       289
      waterway       0.56      0.35      0.43       511
  weed_cluster       0.80      0.90      0.85      2959

      accuracy                           0.76      4431
     macro avg       0.56      0.49      0.52      4431
  weighted avg       0.74      0.76      0.74      4431

